In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os
import csv
from tqdm import tqdm
import pandas as pd

import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions

from torch import optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split


(10000, 1000, 10)


In [2]:
output_dir = os.path.join(".", "output_front_ppo")
subdirs = [f.path for f in os.scandir(output_dir) if f.is_dir()]
subdirs.sort()
combined_arr_f = np.empty((len(subdirs), 1000, 10))
for i, subdir in enumerate(subdirs):
    action_filename = os.path.join(subdir, "action.csv")
    obs_filename = os.path.join(subdir, "obs.csv")
    action_df = pd.read_csv(action_filename,  header=None)
    obs_df = pd.read_csv(obs_filename,  header=None)
    combined_data = pd.concat([action_df, obs_df], axis=1)
    combined_arr_f[i-1] = np.reshape(combined_data.values, (1000, 10))

print(combined_arr_f.shape)
combined_tensor_f = torch.from_numpy(combined_arr_f)

(10000, 1000, 10)


In [3]:
output_dir = os.path.join(".", "output_bw_ppo")
subdirs = [f.path for f in os.scandir(output_dir) if f.is_dir()]
subdirs.sort()

combined_arr_b = np.empty((len(subdirs), 1000, 10))
for i, subdir in enumerate(subdirs):
    action_filename = os.path.join(subdir, "action.csv")
    obs_filename = os.path.join(subdir, "obs.csv")
    action_df = pd.read_csv(action_filename,  header=None)
    obs_df = pd.read_csv(obs_filename,  header=None)
    combined_data = pd.concat([action_df, obs_df], axis=1)
    combined_arr_b[i-1] = np.reshape(combined_data.values, (1000, 10))
print(combined_arr_b.shape)
combined_tensor_b = torch.from_numpy(combined_arr_b)

(10000, 1000, 10)


In [4]:
output_dir = os.path.join(".", "output_right_ppo")
subdirs = [f.path for f in os.scandir(output_dir) if f.is_dir()]
subdirs.sort()

combined_arr_r = np.empty((len(subdirs), 1000, 10))
for i, subdir in enumerate(subdirs):
    action_filename = os.path.join(subdir, "action.csv")
    obs_filename = os.path.join(subdir, "obs.csv")
    action_df = pd.read_csv(action_filename,  header=None)
    obs_df = pd.read_csv(obs_filename,  header=None)
    combined_data = pd.concat([action_df, obs_df], axis=1)
    combined_arr_r[i-1] = np.reshape(combined_data.values, (1000, 10))
print(combined_arr_r.shape)
combined_tensor_r = torch.from_numpy(combined_arr_r)

(10000, 1000, 10)


# VAE code -1

In [23]:
class Encoder(nn.Module):
    def __init__(self, input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim):
        super(Encoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_shape, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim3),
            nn.ReLU(),
            nn.Linear(hidden_dim3, latent_dim * 2)
        )

    def forward(self, x):
        h = self.encoder(x)
        mu, log_var = h.chunk(2, dim=1)
        return mu, log_var, h


class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_shape):
        super(Decoder, self).__init__()

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim3),
            nn.ReLU(),
            nn.Linear(hidden_dim3, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, output_shape),
        )

    def forward(self, z):
        x_reconstructed = self.decoder(z)
        return x_reconstructed


class VAE(nn.Module):
    def __init__(self, input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim):
        super(VAE, self).__init__()

        self.encoder = Encoder(input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim1, hidden_dim2, hidden_dim3, input_shape)

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, log_var, h = self.encoder(x)
        z = self.reparameterize(mu, log_var)
        x_reconstructed = self.decoder(z)
        return x_reconstructed, mu, log_var

In [24]:
datasetf = TensorDataset(combined_tensor_f.float())
datasetb = TensorDataset(combined_tensor_b.float())
datasetr = TensorDataset(combined_tensor_r.float())
batch_size = 512
input_shape = combined_tensor_r.shape[1] * combined_tensor_r.shape[2]

In [25]:
ftrain_size = int(0.7 * len(datasetf))
fvalid_size = int(0.15 * len(datasetf))
ftest_size = len(datasetf) - ftrain_size - fvalid_size
ftrain_dataset, fvalid_dataset, ftest_dataset = random_split(datasetf, [ftrain_size, fvalid_size, ftest_size])

ftrain_loader = DataLoader(ftrain_dataset, batch_size=batch_size, shuffle=True)
fvalid_loader = DataLoader(fvalid_dataset, batch_size=batch_size, shuffle=True)
ftest_loader = DataLoader(ftest_dataset, batch_size=batch_size, shuffle=True)


In [26]:
btrain_size = int(0.7 * len(datasetb))
bvalid_size = int(0.15 * len(datasetb))
btest_size = len(datasetb) - btrain_size - bvalid_size
btrain_dataset, bvalid_dataset, btest_dataset = random_split(datasetb, [btrain_size, bvalid_size, btest_size])

btrain_loader = DataLoader(btrain_dataset, batch_size=batch_size, shuffle=True)
bvalid_loader = DataLoader(bvalid_dataset, batch_size=batch_size, shuffle=True)
btest_loader = DataLoader(btest_dataset, batch_size=batch_size, shuffle=True)


In [27]:
rtrain_size = int(0.7 * len(datasetr))
rvalid_size = int(0.15 * len(datasetr))
rtest_size = len(datasetr) - rtrain_size - rvalid_size
rtrain_dataset, rvalid_dataset, rtest_dataset = random_split(datasetr, [rtrain_size, rvalid_size, rtest_size])

rtrain_loader = DataLoader(rtrain_dataset, batch_size=batch_size, shuffle=True)
rvalid_loader = DataLoader(rvalid_dataset, batch_size=batch_size, shuffle=True)
rtest_loader = DataLoader(rtest_dataset, batch_size=batch_size, shuffle=True)


# 여기서부터

In [28]:
hidden_dim1 = 24  
hidden_dim2 = 12  
hidden_dim3 = 12  
latent_dim = 2  # This gonna *2 in the vae model.
lr = 5e-5  # learning rate
n_epochs = 200  # modify as needed
beta = 0.2
    

# Model, optimizer, and loss function
model = VAE(input_shape, hidden_dim1, hidden_dim2, hidden_dim3, latent_dim)

optimizer = optim.RAdam(model.parameters(), lr=lr)  # Make sure you're using the correct optimizer
loss_fn = nn.MSELoss()  # And the correct loss function


def train(epoch, model, optimizer, loss_fn, train_loader):
    model.train()
    train_loss = 0
    total_recon_loss = 0
    total_kl_div = 0
    for i, batch in enumerate(train_loader):  # using train_loader instead of dataloader
        batch_data = batch[0]  # get the data from the batch
        
        optimizer.zero_grad()

        # Flatten the data
        batch_data = batch_data.view(batch_data.size(0), -1)

        reconstructed_batch, mu, log_var = model(batch_data)

        # Loss: reconstruction loss + KL divergence
        recon_loss = loss_fn(reconstructed_batch, batch_data)
        kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        loss = recon_loss + beta*kl_divergence

        loss.backward()
        train_loss += loss.item()
        total_recon_loss += recon_loss.item()
        total_kl_div += kl_divergence.item()
        optimizer.step()

    avg_loss = train_loss / len(train_loader.dataset)
    avg_recon_loss = total_recon_loss / len(train_loader.dataset)
    avg_kl_div = total_kl_div / len(train_loader.dataset)
#     print(f'====> Epoch: {epoch} Average loss: {avg_loss}, Recon Loss: {avg_recon_loss}, KL Div: {avg_kl_div}')

    return avg_loss, avg_recon_loss, avg_kl_div


In [29]:
bavg_losses = []
bavg_recon_losses = []
bavg_kl_divs = []

# Training
for epoch in range(1, n_epochs + 1):
    bavg_loss, bavg_recon_loss, bavg_kl_div = train(epoch, model, optimizer, loss_fn, btrain_loader)
    bavg_losses.append(bavg_loss)
    bavg_recon_losses.append(bavg_recon_loss)
    bavg_kl_divs.append(bavg_kl_div)


KeyboardInterrupt: 

In [ ]:
ravg_losses = []
ravg_recon_losses = []
ravg_kl_divs = []

# Training
for epoch in range(1, n_epochs + 1):
    ravg_loss, ravg_recon_loss, ravg_kl_div = train(epoch, model, optimizer, loss_fn, rtrain_loader)
    ravg_losses.append(ravg_loss)
    ravg_recon_losses.append(ravg_recon_loss)
    ravg_kl_divs.append(ravg_kl_div)


In [ ]:
favg_losses = []
favg_recon_losses = []
favg_kl_divs = []
for epoch in range(1, n_epochs + 1):
    favg_loss, favg_recon_loss, favg_kl_div = train(epoch, model, optimizer, loss_fn, ftrain_loader)
    favg_losses.append(favg_loss)
    favg_recon_losses.append(favg_recon_loss)
    favg_kl_divs.append(favg_kl_div)


# latent space plot

In [ ]:
encoded_representations_forward = []
encoded_representations_backward = []
encoded_representations_right = []
model.eval()
with torch.no_grad():
    for batch in ftest_loader:
        batch_data = batch[0]
        batch_data = batch_data.view(batch_data.size(0), -1)
        batch_data = batch_data.float()
        _, _, h = model.encoder(batch_data)
        encoded_representations_forward.append(h)

encoded_representations_forward = torch.cat(encoded_representations_forward, dim=0).numpy()

with torch.no_grad():
    for batch in btest_loader:
        batch_data = batch[0]
        batch_data = batch_data.view(batch_data.size(0), -1)
        batch_data = batch_data.float()
        _, _, h = model.encoder(batch_data)
        encoded_representations_backward.append(h)
        
encoded_representations_backward = torch.cat(encoded_representations_backward, dim=0).numpy()

with torch.no_grad():
    for batch in rtest_loader:
        batch_data = batch[0]
        batch_data = batch_data.view(batch_data.size(0), -1)
        batch_data = batch_data.float()
        _, _, h = model.encoder(batch_data)
        encoded_representations_right.append(h)
        
encoded_representations_right = torch.cat(encoded_representations_right, dim=0).numpy()

plt.figure(figsize=(8, 6))
plt.scatter(encoded_representations_forward[:, 0], encoded_representations_forward[:, 2], alpha=0.5, s=2, label='Forward')
plt.scatter(encoded_representations_backward[:, 0], encoded_representations_backward[:, 2], alpha=0.5, s=2, label='Backward')
plt.scatter(encoded_representations_right[:, 0], encoded_representations_right[:, 2], alpha=0.5, s=2, label='Right')

plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('Visualization of latent space')
plt.legend()
plt.show()
